# Image Classification Task V.1

- This task is working on 3 classes of Raw Data Sets classes ( car = 0 , cat = 1 , dog = 2 ). 
- this was Bulit using  keras (based on TensorFlow package ). 

In [13]:
import sys
import os
from os import listdir
from os.path import isfile, join
import numpy as np
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model

### Making Configs variables

In [ ]:
train_data_path = './data/train'
validation_data_path = './data/validation'

"""
Parameters
"""
img_width, img_height = 150, 150
batch_size = 32
epochs = 10
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 3
lr = 0.0004

## Data Modeling

In [4]:
model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5)) # regulization 0.5 
model.add(Dense(classes_num, activation='softmax'))

# Compile model an RmsProp Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Fitting the model 
model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps)


model.save('model.h5')
model.save_weights('weights.h5')

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", input_shape=(150, 150,...)`
/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(data_format="channels_first", pool_size=(2, 2))`


Found 2361 images belonging to 3 classes.
Found 1412 images belonging to 3 classes.


/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:65: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=10, validation_data=<keras_pre..., steps_per_epoch=31, validation_steps=300)`


Epoch 1/10
31/31 [==============================] - 392s 13s/step - loss: 2.2284 - acc: 0.4527 - val_loss: 0.9156 - val_acc: 0.4814

In [18]:
# Checking model and it's overall params.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 75, 75, 64)        8256      
_________________________________________________________________
activation_8 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 75, 37, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 88800)             0         
__________

### Model Evaluation

In [17]:
# dimensions of our images
img_width, img_height = 150, 150

# load the model we saved
model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

mypath = "predict/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
dog_counter = 0 
cat_counter  = 0
car_counter = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0]
    
    if classes == 0:
        print(file + ": " + 'car')
        car_counter += 1
    elif classes == 1 :
        print(file + ": " + 'cat')
        cat_counter += 1
    else:
        print(file + ": " + 'dog')
        dog_counter += 1
print("Total cars :",car_counter)
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

['3.jpg', '5.jpg', '9.jpg', '6.jpg', '7.jpg', 'dog.3.jpg', '1.jpg', '8.jpg', '2.jpg', 'dog.2.jpg', '4.jpg', 'dog.4.jpg']
3.jpg: dog
5.jpg: dog
9.jpg: car
6.jpg: dog
7.jpg: car
dog.3.jpg: dog
1.jpg: dog
8.jpg: car
2.jpg: cat
dog.2.jpg: dog
4.jpg: cat
dog.4.jpg: dog
('Total cars :', 3)
('Total Dogs :', 7)
('Total Cats :', 2)


# You can only run first cell and last one, because it model take a while so I save it in file generator model.h5 so you run last cell will get you evaluation of model